In [ ]:
import wave
import ntpath
import numpy as np
import tflite_runtime.interpreter as tflite

In [ ]:
file_path = 'path/to/file'
file = ntpath.basename(file_path)
real = file.rsplit('.', 1)[0]

ifile = wave.open('path/to/file')
samples = ifile.getnframes()
audio = ifile.readframes(samples)

audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)  
audio_as_np_float32 = audio_as_np_int16.astype(np.float32)  
max_int16 = 2**15  

audio_normalised = audio_as_np_float32 / max_int16
audio_normalised = np.ndarray.astype(audio_normalised, 'float32')

In [ ]:
zero_padding = np.zeros(48000-np.shape(audio_normalised)[0])  # based on the duration the value 48000 will differ
equal_length = np.concatenate([audio_normalised, zero_padding], 0)

In [ ]:
frame_length=255
frame_step=128

w = np.hanning(frame_length+1)
X = np.array([np.fft.rfft(w*equal_length[i:i+frame_length+1]) 
              for i in range(0, len(equal_length)-frame_length, frame_step)])


In [ ]:
X = np.abs(X)

spectrogram = np.reshape(X, [1,374,129,1])
spectrogram = spectrogram.astype(np.float32)

In [ ]:
interpreter = tflite.Interpreter(model_path="path/to/tflite/model")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], spectrogram)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

fnl_out = output_data.flatten()

In [ ]:
data = fnl_out*100

def index(i):
    switcher={
        0:'Backup',
        1:'Connection',
        2:'Device Diagnostic', 
        3:'Device Info',
        4:'Hello PDM',
        5:'No',
        6:'Reboot PDM',
        7:'Reboot to recovery mode',
        8:'Remove USB',
        9:'Run all',
        10:'Start local setup',
        11:'Stop local setup',
        12:'System setup',
        13:'Yes'
        
    }
    return switcher.get(i,"Invalid")

for i in range(len(data)):
    if(data[i]==max(data)):
        print("\nInput audio:", real)
        print("Prediction:", index(i))
        print("Confidence Score", data[i])

In [ ]:
x_p=[]
y_c=['Backup',
   'Connection',
   'Device Diagnostic', 
   'Device Info','Hello PDM',
   'No',
   'Reboot PDM',
   'Reboot to recovery mode',
   'Remove USB',
   'Run all',
   'Start local setup',
   'Stop local setup',
   'System setup','Yes']
for i in range(len(data)):
    x_p.append(data[i])


In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(figsize =(5, 5))
 
# Horizontal Bar Plot
ax.barh(y_c, x_p)
 
# Remove axes splines
for s in ['top', 'bottom', 'left', 'right']:
    ax.spines[s].set_visible(False)
 
# Remove x, y Ticks
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')
 
# Add padding between axes and labels
ax.xaxis.set_tick_params(pad = 5)
ax.yaxis.set_tick_params(pad = 10)
 
# Add x, y gridlines
ax.grid(b = True, color ='black',
        linestyle ='-.', linewidth = 0.5,
        alpha = 0.2)
 
# Show top values
ax.invert_yaxis()
 
# Add annotation to bars
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize = 10, fontweight ='bold',
             color ='grey')
 
# Add Plot Title
ax.set_xlabel("Confidence Score (Percentage)")
ax.set_ylabel("Classes")
ax.set_title('Label Prediction', loc ='left')

# Show Plot
plt.show()